In [1]:
#export
import k1lib as k1, os, json, k1lib, time, dill
import k1lib.cli as cli
from collections import deque
from functools import lru_cache
from contextlib import contextmanager
__all__ = ["sql", "sqldb", "sqltable", "sqlrow", "minio", "s3", "s3bucket", "s3obj", "Redis"]

Connection managers, just so that the really hard problem of making things robust is separated to here. This also enables maximum connection reuses:

In [3]:
#export
psycopg2 = k1.dep("psycopg2", "psycopg2-binary", "https://pypi.org/project/psycopg2/")
class PgConn:
    def __init__(self, p:"PgConns", db:str):
        self.p = p; self.db = db; self.conn = None; self._connect()
    def _connect(self): p = self.p; self.conn = psycopg2.connect(host=p.host, port=p.port, database=self.db, user=p.user, password=p.password); self.conn.autocommit = True
    def queryRaw(self, f): cur = self.conn.cursor(); res = f(cur); cur.close(); return res # absolute barebones, just prepare the cursor and handles all errors before
    def query(self, query, *args, mode=0): # modes: 0 (result), 1 ([res, desc]), 2 (assumes result is a table, then inserts desc to the top row)
        def inner(cur):
            res = None; desc = None; cur.execute(query, args)
            try: res = cur.fetchall()
            except psycopg2.ProgrammingError: pass
            if mode > 0: desc = [x.name for x in cur.description]
            return res if mode == 0 else ((res, desc) if mode == 1 else [desc, *res])
        return self.queryRaw(inner)
    def __repr__(self): return f"<PgConn host={self.p.host}:{self.p.port} db={self.db}>"
class PgConns:
    def __init__(self, host:str, port:int, user:str, password:str):
        self.host = host; self.port = port; self.user = user; self.password = password; self.conns = {}
    def __getitem__(self, key:str):
        if key not in self.conns: self.conns[key] = PgConn(self, key)
        return self.conns[key]
    def __repr__(self): return f"<PgConns host={self.host}:{self.port}>"
class PgManager: # singleton, to efficiently manages all connections from everywhere
    def __init__(self): self.connss = {}
    def __call__(self, host, port, user, password):
        key = (host, port, user, password)
        if key not in self.connss: self.connss[key] = PgConns(host, port, user, password)
        return self.connss[key]
mysqlConn = k1.dep("mysql.connector", "mysql-connector-python", "https://pypi.org/project/mysql-connector-python/")
class MyConn:
    def __init__(self, p:"MyConns", db:str): self.p = p; self.db = db; self.conn = None; self._connect()
    def _connect(self): p = self.p; self.conn = mysqlConn.connect(host=p.host, port=p.port, database=self.db, user=p.user, password=p.password, charset='utf8mb4', collation='utf8mb4_general_ci'); self.conn.autocommit = True
    def queryRaw(self, f): cur = self.conn.cursor(); res = f(cur); cur.close(); return res
    def query(self, query, *args, mode=0):
        def inner(cur):
            res = None; desc = None; cur.execute(query, args)
            res = cur.fetchall()
            if mode > 0: desc = [x[0] for x in cur.description]
            return res if mode == 0 else ((res, desc) if mode == 1 else [desc, *res])
        return self.queryRaw(inner)
    def __repr__(self): return f"<MyConn host={self.p.host}:{self.p.port} db={self.db}>"
class MyConns:
    def __init__(self, host:str, port:int, user:str, password:str):
        self.host = host; self.port = port; self.user = user; self.password = password; self.conns = {}
    def __getitem__(self, key:str):
        if key not in self.conns: self.conns[key] = MyConn(self, key)
        return self.conns[key]
    def __repr__(self): return f"<MyConns host={self.host}:{self.port}>"
class MyManager:
    def __init__(self): self.connss = {}
    def __call__(self, host, port, user, password):
        key = (host, port, user, password)
        if key not in self.connss: self.connss[key] = MyConns(host, port, user, password)
        return self.connss[key]
sqlite3 = k1.dep("sqlite3"); 
class LiConn:
    def __init__(self, conn, fn): self.conn = conn; self.fn = fn
    def queryRaw(self, f): cur = self.conn.cursor(); res = f(cur); self.conn.commit(); cur.close(); return res
    def query(self, query, *args, mode=0):
        def inner(cur):
            res = None; desc = None; cur.execute(query, args); res = cur.fetchall()
            if mode > 0: desc = [x[0] for x in cur.description]
            return res if mode == 0 else ((res, desc) if mode == 1 else [desc, *res])
        return self.queryRaw(inner)
    def __repr__(self): return f"<LiConn fn={self.fn} db='default'>"
class LiConns:
    def __init__(self, fn:str): self.conn = LiConn(sqlite3.connect(fn, check_same_thread=False), fn); self.fn = fn
    def __getitem__(self, key:str): return self.conn
    def __repr__(self): return f"<LiConns fn={self.fn}>"
class LiManager:
    def __init__(self): self.connss = {}
    def __call__(self, fn:str):
        if fn not in self.connss: self.connss[fn] = LiConns(fn)
        return self.connss[fn]
pgM = PgManager(); myM = MyManager(); liM = LiManager()

In [2]:
#export
settings = k1lib.settings
@contextmanager
def mysqlCnf(user, password, host, port):
    fn = f"""[client]\nuser = "{user}"\npassword = "{password or ''}"\nhost = "{host}"\nport = "{port}" """ | cli.file()
    try: yield fn
    finally: os.remove(fn)
rowCaches = set() # k1lib.cache objects for caching table[id]
def cb_rowCache_size(s,v):
    for rC in rowCaches: rC.maxsize = v
def cb_rowCache_timeout(s, v):
    for rC in rowCaches: rC.timeout = v
settings.cred.add("sql", k1lib.Settings()
    .add("mode", "my", env="K1_SQL_MODE")
    .add("host", "127.0.0.1", "host name of db server, or db file name if mode='lite'. Warning: mysql's mysqldump won't resolve domain names, so it's best to pass in ip addresses", env="K1_SQL_HOST")
    .add("port", 3306, env="K1_SQL_PORT")
    .add("user", "admin", sensitive=True, env="K1_SQL_USER")
    .add("password", "admin", sensitive=True, env="K1_SQL_PASSWORD")
    .add("verbose", False, "if True, will print out all executed queries")
    .add("sanitize", False, "if True, will sanitize all string columns with MarkupSafe", env=("K1_SQL_SANITIZE", lambda x: x.lower()[0] == "t"))
    .add("cache", k1lib.Settings()
         .add("size", 300, "Size of the cache, 0 to disable it", cb_rowCache_size, env=("K1_SQL_CACHE_SIZE", lambda x: int(x)))
         .add("timeout", 1, "After this number of seconds, the cached item will expire", cb_rowCache_timeout, env=("K1_SQL_CACHE_TIMEOUT", lambda x: float(x))), "Cache settings for table accesses. I.e table[id]")
, "anything related to sql, used by k1lib.cli.lsext.sql. See docs for that class for more details")
mysqlConn = k1.dep("mysql.connector", "mysql-connector-python", "https://pypi.org/project/mysql-connector-python/")
pgConn = k1.dep("psycopg2", "psycopg2-binary", "https://pypi.org/project/psycopg2/")
sqlite3 = k1.dep("sqlite3"); markupsafe = k1.dep("markupsafe", "MarkupSafe", "https://pypi.org/project/MarkupSafe/")
default = object(); qD = {"my": "`", "pg": "", "lite": ""} # quote dict
def stripMemView(x): return bytes(x) if isinstance(x, memoryview) else x # postgresql returns memory view objects for bytea columns, instead of just raw bytes, so gotta strip it
class alldump(list): pass # dump of all databases
class sql:
    def __init__(self, host=default, port=default, user=default, password=default, mode=default):
        """Creates a connection to a SQL database.
Example::

    s = sql("127.0.0.1") # creates a new sql object. Apparently, mysql and mysqldump on the command line bugs out if you put "localhost". If you put localhost here, it should work fine, but `sql(...) | toBytes()` and other functions that use external programs can deny you access
    s.refresh()          # refreshes connection any time you encounter strange bugs

    s | ls()                                                       # returns List[sqldb], lists out all databases
    s | toBytes()                                                  # returns Iterator[str], dumps every databases. Yes, it's "toBytes", but it has the feeling of serializing whatever the input is, so it's at least intuitive in that way
    "dump.sql" | s                                                 # restores the database using the dump file
    cat("dump.sql") | s                                            # restores the database using the dump file
    db1 = s["db1"]                                                 # returns sqldb, gets database named "db1"
    db1 | ls()                                                     # returns List[sqltable], list out all tables within this database
    db1 | toBytes()                                                # returns Iterator[str], dumps the database
    users = db1["user"]                                            # gets table named "user", short and simple

    db1.query("select * from users")                     # queries the database using your custom query
    db1.query("select * from users where user_id=%s", 3) # queries with prepared statement

    users.info()                                               # prints out the first 10 rows of the table and the table schema
    users.cols                                                 # returns table's columns as List[str]
    len(users)                                                 # returns number of rows
    users.query(...)                                           # can also do a custom query, just like with databases
    users[1]                                                   # grabs user with id 1. Returns None if user doesn't exist
    users[1:10]                                                # grabs users with id from 1 (inclusive) to 10 (exclusive)
    users[1].firstname = "Reimu"                               # sets the first name of user with id 1 to "Reimu"
    users[1] = {"firstname": "Reimu", "lastname": "Hakurei"}   # sets first name and last name of user with id 1
    for user in users: user.firstname = "Reimu"                # loops through all users and change the firstname to "Reimu"
    users.insert(firstname="Yuyuko", lastname="Saigyouji")     # inserts a new row
    users.insertBulk(firstname=["Yuyuko", "Marisa"], lastname=["Saigyouji", "Kirisame"]) # inserts multiple rows at once
    users | toBytes()                                          # dumps this specific table, returns Iterator[str]
    users |  cat() | display()                                 # reads entire table, gets first 10 rows and displays it out
    users | (cat() | head(20)) | display()                     # reads first 20 rows only, then displays the first 10 rows. Query sent is "select * from user limit 20"
    users | (cat() | filt("x == 4", 3) | head(20)) | display() # grabs first 20 rows that has the 4th column equal to 4, then displays the first 10 rows. Query sent is "select user_id, address, balance, age from user where age = 4", assuming the table only has those columns

Philosophy for these methods is that they should be intuitive to look at and interact with, not for performance.
If performance is needed, just write raw queries or don't use a database and use applyCl() instead.

If any of the params here are not specified, they will take on the values from
"settings.cred.sql", which can be initialized from environment variables.
Execute and display/print "settings" in a new cell to see all of them.

:param host: host name, ip address, or file name (in case of sqlite)
:param port: port at the host
:param uesr: database user name. If not specified then fallback to environment variable ``SQL_USER``, then ``USER``
:param password: database password. If not specified then assume database doesn't require one
:param mode: currently supports 3 values: "my" (MySQL), "pg" (PostgreSQL) and "lite" (SQLite)"""
        host = settings.cred.sql.host if host is default else host; port = settings.cred.sql.port if port is default else port; user = settings.cred.sql.user if user is default else user
        password = settings.cred.sql.password if password is default else password; mode = settings.cred.sql.mode if mode is default else mode
        if mode not in ("my", "pg", "lite"): raise Exception(f"Supports only 'my' (MySQL), 'pg' (PostgreSQL) and 'lite' (SQLite) for now, don't support {mode}")
        if mode == "lite": host = os.path.expanduser(host)
        self.host = host; self.port = port; self.user = user or os.environ.get("SQL_USER") or os.environ.get("USER")
        self.password = password or os.environ.get("SQL_PASSWORD"); self.conn:"PgConn" = None; self.mode = mode; self.conn = None; self.refresh()
        self.star = "%s" if mode == "my" or mode == "pg" else "?"
    def queryDesc(self, query, *args, fetch=True, desc=True, commit=True):
        """Executes a query. Returns the resulting table and the description of the table (with column names and whatnot)"""
        if settings.cred.sql.verbose: print(f"query: {query}, args: {args}")
        # ok look, this code looks messy, and there're no guarantees on transactions and whatnot, but there're just so many bugs that only happen after
        # a while that it's pretty annoying! For future readers, if you can implement something less ugly while still not causing long term problems, be my guest
        try:
            cur = self.cursor(); cur.execute(query, args); res = None; desc_ = None
            if fetch:
                try: res = cur.fetchall() # some commands like "use database1", this errors out, but it's perfectly normal behavior
                except: pass
            if desc: desc_ = cur.description
            if commit:
                if self.mode == "pg":
                    try: self.conn.commit() # sometimes postgresql would close the connection. No idea why, so am gonna just catch this
                    except: pass
                else: self.conn.commit()
            cur.close(); return res, desc_
        except: # execute again, hopefully to clear out connection lost errors. Connection is even lost when the db is on the same host! Like, wtf? This happens with mysql only tho
            self.refresh(); cur = self.cursor(); cur.execute(query, args); res = None; desc_ = None
            if fetch:
                try: res = cur.fetchall()
                except: pass
            if desc: desc_ = cur.description
            if commit:
                if self.mode == "pg":
                    try: self.conn.commit() # sometimes postgresql would close the connection. No idea why, so am gonna just catch this
                    except: pass
                else: self.conn.commit()
            cur.close(); return res, desc_
    def query(self, query, *args, **kw):
        """Executes a query. Returns the resulting table (if select query).
Example::

    s.query("insert into users (name, age) values (%s, %s)", "Reimu", 25)
"""
        return self.queryDesc(query, *args, **{**kw, "desc": False})[0]
    def queryMany(self, query, *args):
        """Executes multiple queries at the same time.
Example::

    s.queryMany("insert into users (name, age) values (%s, %s)", [("Reimu", 25), ("Marisa", 26)])
"""
        cur = self.cursor(); cur.executemany(query, *args)
        try: ans = cur.fetchall()
        except: ans = None
        cur.close(); self.conn.commit(); return ans
    def _ls(self):
        if self.mode == "my": return [sqldb(self._copy(e[0]), e[0]) for e in self.query("show databases")]
        elif self.mode == "pg": return [sqldb(self._copy(e[0]), e[0]) for e in self.query("select datname from pg_database where datistemplate=false")]
        elif self.mode == "lite": return [sqldb(self, "default")]
    def __repr__(self): return f"<sql mode={self.mode} host={self._host}>"
    @property
    def _host(self) -> str:
        if self.mode == "lite": return self.host.split(os.sep)[-1]
        else: return f"{self.host}:{self.port}"
    def _cnfCtx(self): return mysqlCnf(self.user, self.password, self.host, self.port)
    def _toBytes(self):
        if self.mode == "my":
            with self._cnfCtx() as fn: k1lib.depCli("mysqldump"); return alldump(None | cli.cmd(f"mysqldump --defaults-file={fn} --single-transaction --hex-blob --all-databases"))
        elif self.mode == "pg": k1lib.depCli("pg_dumpall"); return alldump(None | cli.cmd(f"PGPASSWORD={self.password} pg_dumpall -h {self.host} -p {self.port} -U {self.user}"))
        else: raise Exception(f"All databases dump of mode {self.mode} is not supported yet")
    def __ror__(self, it): # restoring a backup
        if self.mode == "my":
            def restore(fn):
                k1lib.depCli("mysql")
                with self._cnfCtx() as cnfFn: None | cli.cmd(f"mysql --defaults-file={cnfFn} < {fn}") | cli.ignore()
            if isinstance(it, str):     restore(it)
            else: fn = it | cli.file(); restore(fn); os.remove(fn)
        elif self.mode == "pg":
            if isinstance(it, str): fn = it; isStr = True
            elif not isinstance(it, alldump): raise Exception("The sql commands piped in is from a specific database, while you haven't chosen a particular database from this sql connection yet. Do something like `s = sql(...); db = s['some_db_name']; ['some sql'] | db`, instead of `['some sql'] | s` like you're doing now")
            else: fn = it | cli.file(); isStr = False
            try: None | cli.cmd(f"PGPASSWORD={self.password} psql -h {self.host} -p {self.port} -U {self.user} -d postgres < {fn}") | cli.ignore()
            finally:
                if not isStr: os.remove(fn)
        else: raise Exception(f"Restoring database from .sql file of mode {self.mode} is not supported yet")
    def __len__(self): return len(self._ls())
    @lru_cache
    def _getitem(self, idx):
        if isinstance(idx, str):
            lsres = [x for x in self._ls() if x.name == idx]
            if len(lsres) == 1: return lsres[0]
            else: raise Exception(f"Database '{idx}' does not exist, can't retrieve it")
        return self._ls()[idx]
    def __getitem__(self, idx): return self._getitem(idx)
    def __iter__(self): return iter(self._ls())
    def __delitem__(self, idx):
        if isinstance(idx, str):
            if self.mode == "pg": idx = idx.lower()
            lsres = [x for x in self._ls() if x.name == idx]
            if len(lsres) == 1:
                if self.mode == "my": return self.query(f"drop database {idx}")
                elif self.mode == "pg": self._changeDb("postgres"); return self.query(f"drop database {idx} with (force)")
                else: raise Exception("Currently only support dropping databases in mysql and postgresql")
            else: raise Exception(f"Database '{idx}' does not exist, can't delete it")
        else: raise Exception("Only supports deleting with table names. Use this like `del db['some_table']`")
class dbdump(list): pass
class sqldb:
    def __init__(self, sql:sql, name:str):
        """A sql database representation. Not expected to be instatiated by you. See also: :class:`sql`"""
        self.sql = sql; self.name = name
    def query(self, query, *args):
        """Executes a query in this database. Returns the resulting table (if select query).
Example::

    db = ...
    db.query("insert into users (name, age) values (%s, %s)", "Reimu", 25)
"""
        # self.sql._changeDb(self.name)
        return self.sql.query(query, *args)
    def queryDesc(self, query, *args):
        # self.sql._changeDb(self.name);
        return self.sql.queryDesc(query, *args)
    def queryMany(self, query, *args):
        """Executes multiple queries at the same time in this database.
Example::

    db = ...
    db.queryMany("insert into users (name, age) values (%s, %s)", [("Reimu", 25), ("Marisa", 26)])
"""
        # self.sql._changeDb(self.name)
        return self.sql.queryMany(query, *args)
    def _ls(self):
        if self.sql.mode == "my": return [sqltable(self.sql, self, e[0]) for e in self.query(f"show tables")]
        if self.sql.mode == "pg": return [sqltable(self.sql, self, e[0]) for e in self.query(f"select table_name from information_schema.tables where table_schema = 'public'")]
        if self.sql.mode == "lite": return [sqltable(self.sql, self, e[0]) for e in self.query("select name from sqlite_master where type='table'")]
    def __repr__(self): return f"<sqldb host={self.sql._host} db={self.name}>"
    def _toBytes(self):
        if self.sql.mode == "my":
            with self.sql._cnfCtx() as fn: return dbdump(None | cli.cmd(f"mysqldump --defaults-file={fn} --single-transaction --hex-blob --databases {self.name}"))
        elif self.sql.mode == "pg": return dbdump(None | cli.cmd(f"PGPASSWORD={self.sql.password} pg_dump -h {self.sql.host} -p {self.sql.port} -U {self.sql.user} -d {self.name}"))
        else: raise Exception(f"Database dump of mode {self.sql.mode} is not supported yet")
    def __ror__(self, it):
        mode = self.sql.mode; q = qD[mode]
        if mode == "my":
            if isinstance(it, str):
                with open(it, "r") as f: it = f.readlines() # loads file to ram
            return self.sql.__ror__([f"USE {q}{self.name}{q};", *it])
        elif mode == "pg":
            if isinstance(it, str): fn = it; isStr = True
            elif isinstance(it, alldump): self.sql.__ror__(it); return
            else: fn = it | cli.file(); isStr = False
            try: None | cli.cmd(f"PGPASSWORD={self.sql.password} psql -h {self.sql.host} -p {self.sql.port} -U {self.sql.user} -d {self.name} < {fn}") | cli.ignore()
            finally:
                if not isStr: os.remove(fn)
    def __len__(self): return len(self._ls())
    @k1.cache(100, 10)
    def _getitem(self, idx):
        if isinstance(idx, str):
            mode = self.sql.mode
            if mode == "my" or mode == "lite": lsres = [x for x in self._ls() if x.name == idx]
            elif mode == "pg": lsres = [x for x in self._ls() if x.name.lower() == idx.lower()]
            if len(lsres) == 1: return lsres[0]
            else: raise Exception(f"Table '{idx}' does not exist, can't retrieve it")
        return self._ls()[idx]
    def __getitem__(self, idx): return self._getitem(idx)
    def __iter__(self): return iter(self._ls())
    def __delitem__(self, idx):
        if isinstance(idx, str):
            if self.sql.mode == "pg": idx = idx.lower()
            lsres = [x for x in self._ls() if x.name == idx]
            if len(lsres) == 1: return self.query(f"drop table {idx}")
            else: raise Exception(f"Table '{idx}' does not exist, can't delete it")
        else: raise Exception("Only supports deleting with table names. Use this like `del db['some_table']`")
class tbldump(list): pass
class sqltable:
    def __init__(self, sql, sqldb, name:str):
        """A sql table representation. Not expected to be instantiated by you. See also: :class:`sql`"""
        self.sql = sql; self.sqldb = sqldb; self.name = name; self._cols = None; self.__col2Type = None
    def _col2Type(self):
        if self.__col2Type == None: self.__col2Type = {row[0]: row[1].lower() for row in self._describe()[1:]}
        return self.__col2Type
    def _cat(self, ser):
        cols = self.cols; _2 = [] # clis that can't be optimized, stashed away to be merged with ser later on
        q = qD[self.sql.mode]; clis = deque(ser.clis); o1 = None; o2 = None; o3 = [] # cut(), head() and filt() opts
        while len(clis) > 0:
            c = clis.popleft()
            if isinstance(c, cli.filt): _2.append(c); break # TODO: add optimizations for filt
            elif o2 is None and isinstance(c, cli.head):
                if round(c.n) != c.n or c.n < 0 or c.inverted or c.n == None: _2.append(c); break
                else: o2 = f"limit {c.n}"; continue
            elif o1 is None and isinstance(c, cli.cut):
                if isinstance(c.columns, slice): _2.append(c); o1 = 0; continue
                else:
                    o1 = ", ".join([f"{q}{c}{q}" for c in cols | cli.rows(*c.columns)])
                    if len(c.columns) == 1: _2.append(cli.item().all() | cli.aS(list))
            else: _2.append(c); break
        o1 = o1 or ", ".join([f"{q}{c}{q}" for c in cols])
        query = f"select {o1} from {q}{self.name}{q} {o2 or ''}"#; print(f"query: {query}"); return []
        sql = self.sql; return [sqlrow(sql, self, row) for row in self.sqldb.query(query) | cli.serial(*_2, *clis)]
    @property
    def cols(self):
        """Get column names.
Example::

    db.cols # returns List[str], like ["id", "userId", "name", ...]
"""
        if not self._cols: self._cols = self._describe()[1:] | cli.cut({"my": 0, "pg": 0, "lite": 1}[self.sql.mode]) | cli.deref()
        return self._cols
    @lru_cache
    def _describe(self):
        if self.sql.mode == "my": return self.sqldb.query(f"describe `{self.name}`") | cli.insert(["Field", "Type", "Null", "Key", "Default", "Extra"]) | cli.deref()
        if self.sql.mode == "pg": return self.sqldb.query(f"select column_name, data_type, is_nullable, column_default, ordinal_position from information_schema.columns where table_name='{self.name}' order by ordinal_position") | cli.insert(["column_name", "data_type", "is_nullable", "column_default", "ordinal_position"]) | cli.deref()
        if self.sql.mode == "lite": return self.sqldb.query(f"pragma table_info([{self.name}])") | cli.insert(["cid", "name", "type", "notnull", "dflt_value", "pk"]) | cli.deref()
    def insert(self, **kwargs):
        """Inserts a row.
Example::

    table = ...
    table.insert(firstname="Yuyuko", lastname="Saigyouji")
"""
        q = qD[self.sql.mode]; keys = ", ".join([f"{q}{x}{q}" for x in kwargs.keys()]); values = ", ".join([self.sql.star]*len(kwargs))
        mode = self.sql.mode; san = settings.cred.sql.sanitize; vs = []; _typeD = self._col2Type()
        for k, v in kwargs.items():
            if isinstance(v, dict) or _typeD.get(k, "") == "json": vs.append(json.dumps(v))
            elif isinstance(v, (set, tuple)) or _typeD.get(k, "") == "array": vs.append(list(v))
            elif san and isinstance(v, str): vs.append(markupsafe.escape(v))
            else: vs.append(v)
        if mode == "my" or mode == "lite":
            self.query(f"insert into {self.name} ({keys}) values ({values})", *vs)
            if mode == "my": return self[self.query("select last_insert_id()")[0][0]]
            if mode == "lite": return self[self.sql._lastCur.lastrowid]
        elif mode == "pg": return self[self.query(f"insert into {self.name} ({keys}) values ({values}) returning id", *vs)[0][0]]
    def insertBulk(self, **kwargs):
        """Inserts a row.
Example::

    table = ...
    table.insertBulk(firstname=["Yuyuko", "Marisa"], lastname=["Saigyouji", "Kirisame"])
"""
        q = qD[self.sql.mode]; keys = ", ".join([f"{q}{x}{q}" for x in kwargs.keys()]); values = ", ".join([self.sql.star]*len(kwargs))
        mode = self.sql.mode; san = settings.cred.sql.sanitize; vs = []; _typeD = self._col2Type()
        for k, v in kwargs.items():
            if isinstance(v[0], dict) or _typeD.get(k, "") == "json": vs.append([json.dumps(x) for x in v])
            elif isinstance(v[0], (set, tuple)) or _typeD.get(k, "") == "array": vs.append([list(x) for x in v])
            elif san and isinstance(v[0], str): vs.append([markupsafe.escape(x) for x in v])
            else: vs.append(v)
        self.queryMany(f"insert into {self.name} ({keys}) values ({values})", list(vs | cli.T()))
        if self.sql.mode == "my": return self[self.query("select last_insert_id()")[0][0]]
        if self.sql.mode == "lite": return self[self.sql._lastCur.lastrowid]
    def _update(self, _idx, **kwargs):
        """Updates a row.
Example::

    table = ...
    table.update(3, firstname="Youmu", lastname="Konpaku")
"""
        q = qD[self.sql.mode]; p1 = ", ".join([f"{q}{k}{q} = {self.sql.star}" for k in kwargs.keys()])
        san = settings.cred.sql.sanitize; values = []; _typeD = self._col2Type()
        for k, v in kwargs.items():
            if isinstance(v, dict) or _typeD.get(k, "") == "json": values.append(json.dumps(v))
            elif isinstance(v, (set, tuple)) or _typeD.get(k, "") == "array": values.append(list(v))
            elif san and isinstance(v, str): values.append(markupsafe.escape(v))
            else: values.append(v)
        self.query(f"update {self.name} set {p1} where {self.idCol} = {_idx}", *values)
    def info(self, out=False):
        """Preview table.
Example::

    table = ...
    table.info()

:param out: if True, returns a list of lines instead of printing them out"""
        def gen():
            desc = self._describe() | cli.deref(); cols = self.cols; mode = self.sql.mode; q = qD[mode]; s = ", ".join([f"{q}{e}{q}" for e in cols])
            if mode == "my":
                status = self.sqldb.query(f"show table status like '{self.name}'")[0]
                status = f"engine:{status[1]}, #rows(approx):{status[4]}, len(row):{status[5]}, updated:{status[12]}UTC, created:{status[11]}UTC"
            elif mode == "lite":
                nrows = self.query(f"select max(rowid) from {q}{self.name}{q}")[0][0]
                try: ncols = len(self.query(f"select * from {q}{self.name}{q} limit 1")[0])
                except: ncols = "?"
                status = f"#rows(approx):{nrows}, len(row):{ncols}"
            else: nrows = self.sqldb.query(f"select count(*) from {q}{self.name}{q}")[0][0]; status = f"{nrows} rows total"
            idCol = "rowid" if mode == "lite" else self.idCol; print(f"Table `{self.name}` ({status})\n"); self.sqldb.query(f"select {s} from {q}{self.name}{q} order by {idCol} limit 9") | (cli.aS(stripMemView) | cli.aS(repr) | cli.head(50)).all(2) | cli.insert(cols) | cli.display()
            try: # print tails
                print("...\n..."); self.sqldb.query(f"select {s} from {q}{self.name}{q} order by {idCol} desc limit 9")\
                | cli.reverse() | (cli.aS(stripMemView) | cli.aS(repr) | cli.head(50)).all(2) | cli.insert(cols) | cli.display()
            except: pass
            print("\nTable format:"); desc | cli.display(None)
            if mode == "my": print("\nIndexes:"); a, b = self.sqldb.queryDesc(f"show indexes from {q}{self.name}{q}"); a | cli.insert(b | cli.cut(0)) | cli.display()
            elif mode == "pg": print("\nIndexes:"); a,b = self.sqldb.queryDesc(f"SELECT tablename, indexname, indexdef FROM pg_indexes WHERE schemaname = 'public' and tablename = '{self.name}'"); a | cli.insert(b | cli.op().name.all()) | cli.display()
        if out:
            with k1.captureStdout() as out: gen()
            return out()
        else: gen()
    def _ls(self): return self.info()
    def query(self, query, *args):
        """Executes a query in this table's database. Returns the resulting table (if select query).
Example::

    table = ...
    table.query("insert into users (name, age) values (%s, %s)", "Reimu", 25)
"""
        return self.sqldb.query(query, *args)
    def queryDesc(self, query, *args): return self.sqldb.queryDesc(query, *args)
    def queryMany(self, query, *args):
        """Executes multiple queries at the same time in this table's database.
Example::

    table = ...
    table.queryMany("insert into users (name, age) values (%s, %s)", [("Reimu", 25), ("Marisa", 26)])
"""
        return self.sqldb.queryMany(query, *args)
    def select(self, query, *args):
        """Pretty much identical to :meth:`query`, but this postprocess the results
a little bit, and package them into :class:`sqlrow` objects. Example::

    table = ...
    table.select("where name = %s", "Reimu")
"""
        lq = query.strip().lower()
        if lq.startswith("select") and not lq.startswith("select *"): raise Exception(".select() requires the query to start with 'select *', or remove 'select * from table' statement entirely!")
        if not lq.startswith("select"): query = f"select * from {self.name} {query}"
        query = query.replace("select *", f"select {self.scols}").replace("SELECT *", f"select {self.scols}")
        res = self.query(query, *args); sql = self.sql; return [sqlrow(sql, self, e) for e in res]
    def __repr__(self): return f"<sqltable host={self.sql._host} db={self.sqldb.name} table={self.name}>"
    def _toBytes(self):
        if self.sql.mode == "my":
            with self.sql._cnfCtx() as fn: return tbldump(None | cli.cmd(f"mysqldump --defaults-file={fn} --single-transaction --hex-blob {self.sqldb.name} {self.name}"))
        elif self.sql.mode == "pg": return tbldump(None | cli.cmd(f"PGPASSWORD={self.sql.password} pg_dump -h {self.sql.host} -p {self.sql.port} -U {self.sql.user} -d {self.sqldb.name} -t {self.name}"))
        else: raise Exception(f"Table dump of mode {self.sql.mode} is not supported yet")
    def __ror__(self, it): return self.sqldb.__ror__(it)
    def __len__(self): return self.query(f"select count(*) from {self.name}")[0][0]
    def __iter__(self): return self.select("select * from valves")
    @property
    def scols(self): q = qD[self.sql.mode]; return ", ".join([f"{q}{c}{q}" for c in self.cols]) # string columns
    def lookup(self, **kwargs):
        """Convenience function to lookup 1 instance with the specified value.
Example::

    user = users.lookup(firstname="Reimu")
    # multiple columns work too
    user = users.lookup(firstname="Reimu", lastname="Hakurei")
"""
        p1 = " AND ".join([f"{k} = {self.sql.star}" for k in kwargs.keys()])
        res = self.query(f"select {self.scols} from {self.name} where {p1} limit 1", *kwargs.values())
        return None if len(res) == 0 else sqlrow(self.sql, self, res[0])
    @property
    def idCol(self):
        if self.sql.mode == "pg": # cause postgresql kinda don't respect column orders, so if I were to define "id" as the first column, it wouldn't work reliably. So this will just searches for a column called "id". If found, then uses that for indexing operations, else use the first column like my or lite
            return "id" if len([x for x in self.cols if x == "id"]) > 0 else self.cols[0]
        else: return self.cols[0]
    @property
    def idColPos(self): idCol = self.idCol; return [i for i,x in enumerate(self.cols) if x == idCol][0] if self.sql.mode == "pg" else 0
    _cache = k1.cache(settings.cred.sql.cache.size, settings.cred.sql.cache.timeout); rowCaches.add(_cache)
    @_cache
    def __getitem__(self, idx):
        mode = self.sql.mode; idCol = self.idCol; idColPos = self.idColPos
        q = qD[mode]; scols = ", ".join([f"{q}{c}{q}" for c in self.cols])
        if idx is None: return None
        elif isinstance(idx, (int, str)):
            res = self.query(f"select {scols} from {self.name} where {idCol} = {self.sql.star}", idx)
            if len(res) == 0: return None
            return sqlrow(self.sql, self, res[0])
        elif isinstance(idx, slice):
            idxs = list(range(idx.stop if idx.stop is not None else (self.query(f"select max({idCol}) from {self.name}")[0][0]+1))[idx])
            if len(idxs) == 0: return []
            res = self.query(f"select {scols} from {self.name} where {idCol} in ({', '.join([str(x) for x in idxs])})")
            d = {row[idColPos]:row for row in res}; sql = self.sql; return [sqlrow(sql, self, d[idx]) if d.get(idx, None) else None for idx in idxs]
        else:
            idxs = list(idx)
            if all([isinstance(e, int) for e in idxs]): # does not support string because that might lead to sql injection. Can fix, but too lazy
                if len(idxs) == 0: return []
                res = self.query(f"select {scols} from {self.name} where {idCol} in ({', '.join([str(e) for e in idx])})")
                d = {row[idColPos]:row for row in res}; sql = self.sql; return [sqlrow(sql, self, d[idx]) if d.get(idx, None) else None for idx in idxs]
            raise Exception("Only support table indexing of integers or strings or slices or list[int]")
    def __setitem__(self, idx, value):
        if not isinstance(value, dict): raise Exception("Only accepts setting elements with dicts")
        if self[idx] is None: raise Exception(f"Can't set element with id {idx}, it doesn't seem to exist. Use table.insert(col1=value1, col2=value2) instead")
        self._update(idx, **value)
    def __iter__(self): return iter(self | cli.cat())
    def __delitem__(self, idx):
        idCol = self.idCol; idColPos = self.idColPos
        if isinstance(idx, (int, str)):
            res = self[idx]
            if res: self.query(f"delete from {self.name} where {idCol} = {self.sql.star}", res[idColPos])
        elif isinstance(idx, slice):
            res = self[idx]; ids = ", ".join([row[idColPos] for row in res if row])
            if len(res) > 0: self.query(f"delete from {self.name} where {idCol} in ({ids})")
        else: raise Exception("Only support table indexing of integers or strings or slices")
class sqlrow:
    def __init__(self, sql, sqltable, row):
        row = [stripMemView(x) for x in row]
        self._ab_sentinel = True; self._sql = sql; self._sqltable = sqltable; self._row = row
        self._data = {k:v for k,v in zip(sqltable.cols, row)}; self._ab_sentinel = False
        # if sql.mode == "pg": self.__dict__.update({k.lower():v for k,v in zip(sqltable.cols, row)}) # add case insensitive cases
    def __getitem__(self, idx): return self._row[idx]
    def __getattr__(self, attr):
        if attr in self._data: return self._data[attr]
        if attr.lower() in self._data and self._sql.mode == "pg": return self._data[attr.lower()]
        return self.__dict__[attr]
    def __setattr__(self, attr, value):
        if attr == "_ab_sentinel": self.__dict__[attr] = value
        else:
            if self._ab_sentinel: self.__dict__[attr] = value; return
            attr = attr.lower() if self._sql.mode == "pg" else attr
            if attr not in self._sqltable.cols: self.__dict__[attr] = value
            else:
                self._sqltable._update(self._row[self._sqltable.idColPos], **{attr: value}); self.__dict__[attr] = value
                idx = {x:y for x,y in zip(self._sqltable.cols, range(len(self._row)))}[attr]
                row = list(self._row); row[idx] = value; self._row = row; self._data[attr] = value
    def json(self): return self._data
    def __len__(self): return len(self._row)
    def __repr__(self):
        ans = []
        for k, elem in self._data.items():
            if isinstance(elem, str): ans.append(f"{k}=({len(elem)} len) {json.dumps(elem[:100])}..." if len(elem) > 100 else f"{k}=({len(elem)} len) {json.dumps(elem)}")
            elif isinstance(elem, bytes): ans.append(f"{k}=({len(elem)} len) {elem[:100]}..." if len(elem) > 100 else f"{k}=({len(elem)} len) {elem}")
            else: ans.append(f"{k}={elem}")
        return "(" + f", ".join(ans) + ")"
settings.cli.atomic.baseAnd = (*settings.cli.atomic.baseAnd, sql, sqldb, sqltable, sqlrow)

In [10]:
s = sql("localhost", user="", password=None); s.refresh()
assert s | cli.ls() | cli.shape(0) > 0
assert s | cli.ls() | cli.filt("x.name == 'truckbux'") | cli.shape(0) > 0
db1 = s | cli.ls() | cli.grep("truckbux") | cli.item()
t1 = db1 | cli.ls() | cli.grep("noti") | cli.item(); t1.info(); t1.cols
a = t1.query("select id from notification") | cli.shape(0)
b = t1 | cli.cat() | cli.shape(0)
assert a == b; assert a > 1000
assert t1[1].app_type == "USER"

Table `notification` (engine:InnoDB, #rows(approx):121245, len(row):125, updated:NoneUTC, created:2021-04-27 02:40:48UTC)

notification_id   id   app_type   notification_description                             create_date                                  is_visited   is_active   
1                 2    'USER'     b'Nice! Your order TB-78593DF has successfully bee   datetime.datetime(2017, 10, 19, 4, 34, 14)   1            1           
2                 2    'USER'     b'Nice! Your order TB-4EC736F has successfully bee   datetime.datetime(2017, 10, 19, 4, 35, 20)   1            1           
3                 2    'USER'     b'Congrats! Your order TB-78593DF has been accepte   datetime.datetime(2017, 10, 19, 4, 35, 52)   1            1           
4                 2    'USER'     b'Ayee! Your order TB-78593DF is ready to go. Come   datetime.datetime(2017, 10, 19, 4, 35, 58)   1            1           
5                 2    'USER'     b'Order TB-78593DF was picked up at 12:36am. Enjoy   

In [4]:
s = sql("localhost", 5432, "postgres", "postgres", mode="pg")
t = s | cli.ls() | cli.item() | cli.ls() | cli.item()
assert t.query("select * from pg_statistic") | cli.shape(0) > 0
t.info(); t | (cli.cat() | cli.head()) | cli.display()

Table `pg_statistic` (406 rows total)

starelid   staattnum   stainherit   stanullfrac   stawidth   stadistinct   stakind1   stakind2   stakind3   stakind4   stakind5   staop1   staop2   staop3   staop4   staop5   stacoll1   stacoll2   stacoll3   stacoll4   stacoll5   stanumbers1                                          stanumbers2    stanumbers3      stanumbers4   stanumbers5   stavalues1                                           stavalues2                                           stavalues3   stavalues4   stavalues5   
1247       1           False        0.0           4          -1.0          2          3          0          0          0          609      609      0        0        0        0          0          0          0          0          None                                                 [0.9784336]    None             None          None          '{16,22,28,81,199,273,604,701,790,1002,1009,1015,1   None                                                 None         None      

In [5]:
s = sql("/home/kelvin/ssd/data/wikidata/wikidata.db", mode="lite")
t = s | cli.ls() | cli.item() | cli.ls() | cli.item()
assert t.query("select * from wikidata limit 10") | cli.shape(0) == 10
t.info(); t | (cli.cat() | cli.head()) | cli.display()

Table `wikidata` (70392379 rows total)

idx   type     id        pageid   ns   title     lastrevid    label                 desc                                                 
0     'item'   'Q31'     127      0    'Q31'     1713980862   'Belgium'             'country in western Europe since 1830'               
1     'item'   'Q8'      134      0    'Q8'      1717439938   'happiness'           'mental or emotional state of well-being character   
2     'item'   'Q23'     136      0    'Q23'     1721165431   'George Washington'   'president of the United States from 1789 to 1797'   
3     'item'   'Q24'     137      0    'Q24'     1713446920   'Jack Bauer'          'character from the television series 24'            
4     'item'   'Q42'     138      0    'Q42'     1717100210   'Douglas Adams'       'English science fiction writer and humourist (195   
5     'item'   'Q1868'   142      0    'Q1868'   1710866440   'Paul Otlet'          'Belgian author, librarian and colonial thinker'

In [6]:
from k1lib.imports import *
s = sql("/home/kelvin/ssd/data/wikidata/wikidata.db", mode="lite") | ls() | item(); s | ls()

2024-03-04 15:15:40,967	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-03-04 15:15:40,975	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


[<sqltable host=wikidata.db db=default table=wikidata>,
 <sqltable host=wikidata.db db=default table=descIndex>,
 <sqltable host=wikidata.db db=default table=labelIndex>,
 <sqltable host=wikidata.db db=default table=subDescIndex>,
 <sqltable host=wikidata.db db=default table=subLabelIndex>]

In [7]:
s | ls() | apply(op().info(True) | (item() | aS(fmt.h, 3)) & (~head(2) | join("\n") | aS(fmt.pre)) | join("\n")) | join("\n") | aS(IPython.display.HTML)

In [3]:
#export
boto3 = k1.dep("boto3") # if below so that the tests would run since I have loaded the creds from my startup.py
if "minio" not in settings.cred.__dict__: settings.cred.add("minio", k1lib.Settings().add("host", "https://localhost:9000", env="K1_MINIO_HOST").add("access_key", "", sensitive=True, env="K1_MINIO_ACCESS_KEY").add("secret_key", "", sensitive=True, env="K1_MINIO_SECRET_KEY"), "anything related to minio buckets, used by k1lib.cli.lsext.minio")
def minio(host=None, access_key=None, secret_key=None) -> "s3":
    """Convenience function that constructs a :class:`s3` object but focused on minio.
If the params are not specified, then it takes on the values in the settings
"settings.cred.minio". Example::

    s = minio() # returns s3 instance
    s | ls()    # list all buckets, all other normal operations

:param host: looks like "http://localhost:9000"
"""
    return s3(boto3.client('s3',
            endpoint_url=host or settings.cred.minio.host,
            aws_access_key_id=access_key or settings.cred.minio.access_key,
            aws_secret_access_key=secret_key or settings.cred.minio.secret_key,
            region_name='us-west-2'))
class s3:
    def __init__(self, client):
        """Represents an S3 client.
Example::

    client = boto3.client("s3", ...)            # put your credentials and details here
    db = s3(client)                             # creates an S3 manager
    db | ls()                                   # lists all buckets accessible

    bucket = db | ls() | item()                 # grabs the first bucket, returns object of type s3bucket
    bucket = db["bucket-name"]                  # or you can instantiate the bucket directly
    bucket | ls()                               # lists all objects within this bucket
    bucket | ls() | grep("\\.so")               # grabs all .so files from the bucket

    obj = bucket | ls() | item()                # grabs the first object within this bucket, returns object of type s3obj
    obj = bucket["some_key"]                    # or, grab a specific object
    obj.key, obj.size, obj.lastModified         # some fields directly accessible
    del bucket["some_key"]                      # deletes the object from the bucket

    obj = "abc\ndef" | bucket.upload("somekey") # uploads a file by piping the contents in, returns s3obj
    obj = b"abc\ndef"| bucket.upload()          # same as above, but with an auto-generated key

This mostly offers interoperability with ls() and cat(), so that you can
write relatively intuitive code, but fundamentally provides no upsides

:param client: boto3 client"""
        self.client = client; self._cachedLs = None
    def _ls(self): self._cachedLs = {x["Name"]:s3bucket(self.client, x["Name"]) for x in self.client.list_buckets()["Buckets"]}; return list(self._cachedLs.values())
    def __len__(self):
        if self._cachedLs is None: self._ls()
        return len(self._cachedLs)
    def __getitem__(self, name):
        if self._cachedLs is None: self._ls()
        if name not in self._cachedLs: self._ls()
        if name not in self._cachedLs: raise Exception(f"Bucket '{name}' doesn't exist")
        return self._cachedLs[name]
    def __repr__(self): return f"<kaws.s3 client>"
_s3bucketUploader_autoInc = k1lib.AutoIncrement(prefix=f"_key_{round(time.time()*1000)}_")
class s3bucketUploader(cli.BaseCli):
    def __init__(self, client, bucket:"s3bucket", key:str):
        """Uploads the data piped in, uploads, then return a :class:`s3obj`"""
        self.client = client; self.bucket = bucket; self.key = key
    def __ror__(self, it):
        if isinstance(it, (bytes, str)): self.client.put_object(Bucket=self.bucket.name, Key=self.key, Body=(it if isinstance(it, bytes) else it.encode())) # first path is more direct and doesn't have to save to a file first
        else: tmpFile = it | cli.file(); self.client.upload_file(tmpFile, self.bucket.name, self.key); os.remove(tmpFile)
        return self.bucket[self.key]
    def __repr__(self): return f"<s3bucketUploader bucket.name='{self.bucket.name}' key='{self.key}'>"
class s3bucket:
    def __init__(self, client, name:str):
        """Represents an S3 bucket.
Example::

    client = ...
    db = s3(client)
    bucket = db["bucket-name-here"]

See also: :class:`s3`"""
        self.client = client; self.name = name; self._cachedLs = None
    def _ls(self):
        client = self.client; name = self.name;
        self._cachedLs = {data["Key"]:s3obj(client, name, data) for data in self.client.list_objects(Bucket=name).get("Contents", [])}
        return list(self._cachedLs.values())
    def __len__(self):
        if self._cachedLs is None: self._ls()
        return len(self._cachedLs)
    def upload(self, key:str=None):
        """Uploads some content to s3.
Example::

    bucket = ...
    obj = "abc\ndef"  | bucket.upload() # uploads some text
    obj = b"abc\ndef" | bucket.upload() # uploads some bytes

This works with whatever you can pipe into :class:`~k1lib.cli.output.file`. After uploading,
you will receive a :class:`s3obj` object.

:param key: if not specified, will auto generate a key"""
        if key is None: key = _s3bucketUploader_autoInc()
        return s3bucketUploader(self.client, self, key)
    def __repr__(self): return f"<s3bucket name='{self.name}'>"
    def __getitem__(self, key:str):
        res = self.client.head_object(Bucket=self.name, Key=key)
        data = {"Key": key, "LastModified": res["LastModified"], "Size": int(res["ResponseMetadata"]["HTTPHeaders"]["content-length"]), "StorageClass": None}
        return s3obj(self.client, self.name, data)
    def __delitem__(self, key:str): self.client.delete_object(Bucket=self.name, Key=key)
class s3obj:
    def __init__(self, client, bucket:str, data):
        """Represents an S3 object. Not intended to be instantiated directly.
See also: :class:`s3`"""
        self.client = client; self.bucket = bucket
        self.key  = data["Key"];  self.lastModified = data["LastModified"] | cli.toUnix()
        self.size = data["Size"]; self.storageClass = data["StorageClass"]
    def __repr__(self): return f"<s3obj bucket='{self.bucket}' key='{self.key}' size='{k1lib.fmt.size(self.size)}' lastModified='{self.lastModified | cli.toIso()}'>"
    def _cat(self, kwargs):
        sB = kwargs["sB"]; eB = kwargs["eB"]
        if eB < 0: eB = self.size
        res = self.client.get_object(Bucket=self.bucket, Key=self.key, Range=f'bytes={sB}-{eB-1}')["Body"].read()
        if kwargs["text"]: return res.decode().split("\n")
        if kwargs["chunks"]: return res | cli.batched(settings.cli.cat.chunkSize, True)
        return res

In [45]:
assert minio() | ls() | shape(0)
m = minio()["main"]
assert m | ls() | shape(0)
["abc", "def1"] | m.upload("abc")
assert m["abc"] | cat(text=False) == b"abc\ndef1\n"

In [63]:
#export
redis = k1lib.dep("redis")
_redisAutoInc = k1lib.AutoIncrement(prefix=f"_redis_msg_{round(time.time())}_")
settings.cred.add("redis", k1lib.Settings()
    .add("host", "localhost", "location of the redis server", env="K1_REDIS_HOST")
    .add("port", 6379, "port the redis server use", env=("K1_REDIS_PORT", int))
    .add("expires", 120, "seconds before the message deletes itself. Can be float('inf'), or 'inf' for the env variable", env=("K1_REDIS_EXPIRES", lambda x: float(x.lower()))),
"anything related to redis, used by k1lib.cli.lsext.Redis")
class Redis:
    def __init__(self, host=None, port=None, **kwargs):
        """Connects to Redis server.
Example::

    r = Redis("localhost", 6379)
    r["abc"] = {"some": "json", "number": 4} # sets value
    r["abc"]                                 # reads value, returns python object

    key = r("some message")                  # sets value with auto generated key, returns that key
    r[key]                                   # reads that value, returns "some message"
    key = r("some message", 60)              # sets value with auto generated key, with expires amount, returns that key

You can actually leave the constructors empty, as it will automatically pull the data from
"settings.cred.redis.host", which pulls from the env variable "K1_REDIS_HOST". So it can
be as short as this::

    r = Redis()
    r["abc"] = {"some": "json", "number": 4} # sets value

Internally, all objects are pickled and unpickled, so you can transport pretty much
any Python object. Also, by default, each message will expire after 60s, You can
adjust this via "settings.cred.redis.expires" setting.

This is really only meant as a quick way for different servers to communicate
with each other. If your use case is different, don't use this.

:param host: host name of the redis server
:param port: port the redis server uses
:param kwargs: extra kwargs sent to :class:`redis.Redis`"""
        self.host = host or settings.cred.redis.host; self.port = port or settings.cred.redis.port
        self.client = redis.Redis(host=self.host, port=self.port, **kwargs)
    def __getitem__(self, i): res = self.client.get(i); return None if res is None else dill.loads(res)
    def __setitem__(self, i, v, ex=None): ex = ex or settings.cred.redis.expires; self.client.set(i, dill.dumps(v), **({"ex":ex} if ex < float("inf") else {}))
    def __call__(self, v, ex=None) -> str: i = _redisAutoInc(); self.__setitem__(i, v, ex); return i
    def __repr__(self): return f"<Redis host='{self.host}:{self.port}'>"

In [66]:
r = Redis(port=6380)
r["abc"] = {"some": "json", "number": 4}
assert r["abc"] == {"some": "json", "number": 4}; r

<Redis host='localhost:6380'>

In [2]:
!../../export.py cli/lsext --upload=True

./export started up - /home/quang/miniforge3/bin/python
----- exportAll
16200   0   61%   
10476   1   39%   
Found existing installation: k1lib 1.7
Uninstalling k1lib-1.7:
  Successfully uninstalled k1lib-1.7
DEPRECATION: Loading egg at /home/quang/miniforge3/lib/python3.12/site-packages/aigu-0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Looking in indexes: https://pypi.org/simple, http://10.104.0.3:3141/
Processing /home/quang/k1lib
  Preparing metadata (setup.py) ... done
  Created wheel for k1lib: filename=k1lib-1.7-py3-none-any.whl size=5108463 sha256=f19483af96169fe0ac8440fb708a0c1d02e5e2316101c869aeab7ee20917092e
  Stored in directory: /tmp/pip-ephem-wheel-cache-0pztncl5/wheels/b5/32/67/e20c84dce16d707fb881c12d405f70adfaa36fe7dae9021380
Successfully built k1lib


In [ ]:
!../../export.py cli/lsext

2024-03-12 18:58:25,470	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-03-12 18:58:25,480	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15713   0   61%   
10075   1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.6.2
Uninstalling k1lib-1.6.2:
  Successfully uninstalled k1lib-1.6.2
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdi

In [6]:
!../../export.py cli/lsext --bootstrap=True

./export started up - /home/quang/miniconda3/envs/torch/bin/python3
----- bootstrapping
Current dir: /home/quang/k1lib, /home/quang/k1lib/k1lib/cli/../../export.py
Found existing installation: k1lib 1.7
Uninstalling k1lib-1.7:
  Successfully uninstalled k1lib-1.7
Looking in indexes: https://pypi.org/simple, http://10.104.0.3:3141/
Processing /home/quang/k1lib
  Preparing metadata (setup.py) ... done
  Created wheel for k1lib: filename=k1lib-1.7-py3-none-any.whl size=5104787 sha256=7db92e462ca7d4ab05265cd520468a6a1bd9a10438052c3377f05961235b0554
  Stored in directory: /tmp/pip-ephem-wheel-cache-mymrehz5/wheels/11/94/07/711323eb4091c7ef1b180ccc3793fc75a96521821bdd2932ac
Successfully built k1lib
